In [1]:
import argparse
import os
import os.path as osp

import numpy as np
import torch
print(torch.cuda.is_available())

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import network
import loss
import pre_process as prep
import lr_schedule
from pre_process import ImageList, image_classification_test
import copy
import random



if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Code for RSDA-MSTN')
    parser.add_argument('--gpu_id', type=str, nargs='?', default='0', help="device id to run")
    parser.add_argument('--source', type=str, default='amazon',choices=["amazon", "dslr","webcam"])
    parser.add_argument('--target', type=str, default='dslr', choices=["amazon", "dslr", "webcam"])
    parser.add_argument('--test_interval', type=int, default=50, help="interval of two continuous test phase")
    parser.add_argument('--snapshot_interval', type=int, default=1000, help="interval of two continuous output model")
    parser.add_argument('--lr', type=float, default=0.001, help="learning rate")
    parser.add_argument('--stages', type=int, default=6, help="training stages")
    args = parser.parse_args([])
    s_dset_path = 'data/office/' + args.source + '_list.txt' #'../../data/office/' + args.source + '_list.txt'
    t_dset_path = 'data/office/' + args.target + '_list.txt' #'../../data/office/' + args.target + '_list.txt'

    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_id
    config = {}
    config["source"] = args.source
    config["target"] = args.target
    config["gpu"] = args.gpu_id
    config["test_interval"] = args.test_interval
    config["snapshot_interval"] = args.snapshot_interval
    config["output_for_test"] = True
    config["output_path"] = "snapshot/init"
    if not osp.exists(config["output_path"]):
        os.makedirs(config["output_path"])
    config["out_file"] = open(osp.join(config["output_path"],args.source+"_"+args.target+ "no_transfer_log.txt"), "w")

    config["prep"] = {'params':{"resize_size":256, "crop_size":224}}
    config["network"] = {"name":network.ResNet50, \
            "params":{"new_cls":True,"feature_dim":256,"class_num":31} }
    config["optimizer"] = {"type":optim.SGD, "optim_params":{'lr':args.lr, "momentum":0.9, \
                           "weight_decay":0.0005, "nesterov":True}, "lr_type":"inv", \
                           "lr_param":{"lr":args.lr, "gamma":0.001, "power":0.75} }
    config["data"] = {"source":{"list_path":s_dset_path, "batch_size":36}, \
                      "target":{"list_path":t_dset_path, "batch_size":36}, \
                      "test":{"list_path":t_dset_path, "batch_size":72}}
    config["out_file"].flush()
    if config["source"] == "amazon" and config["target"] == "dslr":
        config["iterations"] = 2000
        seed = 0
    elif config["source"] == "amazon" and config["target"] == "webcam":
        config["iterations"] = 2000
        seed = 0
    else:
        config["iterations"] = 4000
        seed = 1


    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

    config["out_file"].write('\n--- initialization ---\n')
    source = config["source"]
    target = config["target"]
    prep_dict = {}
    prep_dict["source"] = prep.image_train(**config["prep"]['params'])
    prep_dict["target"] = prep.image_train(**config["prep"]['params'])

    prep_dict["test"] = prep.image_test(**config["prep"]['params'])

    ## prepare data
    dsets = {}
    dset_loaders = {}
    data_config = config["data"]
    train_bs = data_config["source"]["batch_size"]
    test_bs = data_config["test"]["batch_size"]
    dsets["source"] = ImageList(open(data_config["source"]["list_path"]).readlines(), \
                                transform=prep_dict["source"])
    dset_loaders["source"] = DataLoader(dsets["source"], batch_size=train_bs, \
                                        shuffle=True, num_workers=0, drop_last=True)
    dsets["target"] = ImageList(open(data_config["target"]["list_path"]).readlines(), \
                                transform=prep_dict["target"])
    dset_loaders["target"] = DataLoader(dsets["target"], batch_size=train_bs, \
                                        shuffle=True, num_workers=0, drop_last=True)

    dsets["test"] = ImageList(open(data_config["test"]["list_path"]).readlines(), \
                              transform=prep_dict["test"])
    dset_loaders["test"] = DataLoader(dsets["test"], batch_size=test_bs, \
                                      shuffle=False, num_workers=0)

    class_num = config["network"]["params"]["class_num"]

    ## set base network
    net_config = config["network"]
    base_network = net_config["name"](**net_config["params"])
    base_network = base_network.cuda()

    ## add additional network for some methods
    ad_net = network.AdversarialNetwork(base_network.output_num(), 1024)
    ad_net = ad_net.cuda()

    gpus = config['gpu'].split(',')
    if len(gpus) > 1:
        ad_net = nn.DataParallel(ad_net)
        base_network = nn.DataParallel(base_network)

    parameter_classifier = [base_network.get_parameters()[1]]
    parameter_feature = base_network.get_parameters()[0:1] + ad_net.get_parameters()

    ## set optimizer
    optimizer_config = config["optimizer"]
    optimizer_classfier = optimizer_config["type"](parameter_classifier, \
                                                   **(optimizer_config["optim_params"]))
    optimizer_feature = optimizer_config["type"](parameter_feature, \
                                                 **(optimizer_config["optim_params"]))
    param_lr = []
    for param_group in optimizer_feature.param_groups:
        param_lr.append(param_group["lr"])
    param_lr.append(optimizer_classfier.param_groups[0]["lr"])
    schedule_param = optimizer_config["lr_param"]
    lr_scheduler = lr_schedule.schedule_dict[optimizer_config["lr_type"]]

    ## train
    len_train_source = len(dset_loaders["source"])
    len_train_target = len(dset_loaders["target"])
    best_acc = 0.0
    best_model = copy.deepcopy(base_network)

    Cs_memory=torch.zeros(class_num,256).cuda()
    Ct_memory=torch.zeros(class_num,256).cuda()


    for i in range(config["iterations"]):
        if i % config["test_interval"] == config["test_interval"] - 1:
            base_network.train(False)
            temp_acc = image_classification_test(dset_loaders,base_network)
            temp_model = base_network
            if temp_acc > best_acc:
                best_acc = temp_acc
                best_model = copy.deepcopy(temp_model)
            log_str = "iter: {:05d}, \t precision: {:.4f},\t best_acc:{:.4f}".format(i, temp_acc, best_acc)
            config["out_file"].write(log_str + "\n")
            config["out_file"].flush()
            print(log_str)
        if (i + 1) % config["snapshot_interval"] == 0:
            if not os.path.exists("save/init_model"):
                os.makedirs("save/init_model")
            torch.save(best_model, 'save/init_model/' + source + '_' + target + 'no_transfer.pkl')

        ## train one iter
        base_network.train(True)
        ad_net.train(True)
        optimizer_classfier = lr_scheduler(optimizer_classfier, i, **schedule_param)
        optimizer_feature = lr_scheduler(optimizer_feature, i, **schedule_param)

        if i % len_train_source == 0:
            iter_source = iter(dset_loaders["source"])
        if i % len_train_target == 0:
            iter_target = iter(dset_loaders["target"])
        inputs_source, labels_source = iter_source.next()
        inputs_target, labels_target = iter_target.next()
        inputs_source, inputs_target, labels_source = inputs_source.cuda(), inputs_target.cuda(), labels_source.cuda()
        features_source, outputs_source = base_network(inputs_source)
        features_target, outputs_target = base_network(inputs_target)
        pseu_labels_target=torch.argmax(outputs_target,dim=1)
        
        loss_sm,Cs_memory,Ct_memory=loss.SM(features_source,features_target,labels_source,pseu_labels_target,
                                            Cs_memory,Ct_memory)
        gamma=network.calc_coeff(i)
        classifier_loss = nn.CrossEntropyLoss()(outputs_source, labels_source)

        loss_total = classifier_loss  + gamma * (loss_sm)

        optimizer_classfier.zero_grad()
        optimizer_feature.zero_grad()

        loss_total.backward()
        optimizer_feature.step()
        optimizer_classfier.step()

        print('step:{: d},\t,class_loss:{:.4f},\t,sm:{:.2f}'
              ''.format(i, classifier_loss.item(),loss_sm.item()))
        Cs_memory.detach_()
        Ct_memory.detach_()

True
step: 0,	,class_loss:3.6366,	,sm:28.31
step: 1,	,class_loss:3.4498,	,sm:24.96
step: 2,	,class_loss:3.4750,	,sm:24.05
step: 3,	,class_loss:3.5438,	,sm:22.64
step: 4,	,class_loss:3.4941,	,sm:22.27
step: 5,	,class_loss:3.5317,	,sm:21.71
step: 6,	,class_loss:3.4979,	,sm:21.59
step: 7,	,class_loss:3.5102,	,sm:20.13
step: 8,	,class_loss:3.3166,	,sm:20.00
step: 9,	,class_loss:3.3018,	,sm:19.27
step: 10,	,class_loss:3.3440,	,sm:18.31
step: 11,	,class_loss:3.3863,	,sm:18.40
step: 12,	,class_loss:3.3420,	,sm:16.18
step: 13,	,class_loss:3.3388,	,sm:16.06
step: 14,	,class_loss:3.2664,	,sm:15.75
step: 15,	,class_loss:3.2584,	,sm:15.61
step: 16,	,class_loss:3.2211,	,sm:15.55
step: 17,	,class_loss:3.2312,	,sm:14.15
step: 18,	,class_loss:3.2178,	,sm:12.02
step: 19,	,class_loss:3.0690,	,sm:11.23
step: 20,	,class_loss:3.2723,	,sm:11.04
step: 21,	,class_loss:3.1677,	,sm:10.93
step: 22,	,class_loss:3.0478,	,sm:9.88
step: 23,	,class_loss:3.0264,	,sm:9.54
step: 24,	,class_loss:2.9835,	,sm:7.34
step: 25

step: 202,	,class_loss:0.6635,	,sm:3.37
step: 203,	,class_loss:0.7089,	,sm:3.33
step: 204,	,class_loss:0.6845,	,sm:3.41
step: 205,	,class_loss:0.8719,	,sm:3.47
step: 206,	,class_loss:0.6840,	,sm:3.31
step: 207,	,class_loss:0.6982,	,sm:3.12
step: 208,	,class_loss:0.7324,	,sm:3.26
step: 209,	,class_loss:0.7331,	,sm:3.34
step: 210,	,class_loss:0.8649,	,sm:3.43
step: 211,	,class_loss:0.6322,	,sm:3.56
step: 212,	,class_loss:0.6750,	,sm:3.50
step: 213,	,class_loss:0.7921,	,sm:3.42
step: 214,	,class_loss:0.5645,	,sm:3.25
step: 215,	,class_loss:0.6604,	,sm:3.28
step: 216,	,class_loss:0.7153,	,sm:3.42
step: 217,	,class_loss:0.9975,	,sm:3.53
step: 218,	,class_loss:0.3986,	,sm:3.24
step: 219,	,class_loss:0.6506,	,sm:3.29
step: 220,	,class_loss:0.6948,	,sm:3.30
step: 221,	,class_loss:0.5547,	,sm:3.10
step: 222,	,class_loss:0.7448,	,sm:3.33
step: 223,	,class_loss:0.6639,	,sm:3.43
step: 224,	,class_loss:0.7611,	,sm:3.31
step: 225,	,class_loss:0.7236,	,sm:3.12
step: 226,	,class_loss:0.6510,	,sm:3.27


step: 402,	,class_loss:0.1779,	,sm:1.77
step: 403,	,class_loss:0.3629,	,sm:1.89
step: 404,	,class_loss:0.2214,	,sm:1.82
step: 405,	,class_loss:0.4453,	,sm:1.81
step: 406,	,class_loss:0.4101,	,sm:1.82
step: 407,	,class_loss:0.5123,	,sm:1.87
step: 408,	,class_loss:0.5550,	,sm:1.78
step: 409,	,class_loss:0.4149,	,sm:1.78
step: 410,	,class_loss:0.3155,	,sm:1.78
step: 411,	,class_loss:0.5182,	,sm:1.81
step: 412,	,class_loss:0.4257,	,sm:1.83
step: 413,	,class_loss:0.4114,	,sm:1.86
step: 414,	,class_loss:0.2855,	,sm:1.76
step: 415,	,class_loss:0.3698,	,sm:1.84
step: 416,	,class_loss:0.2568,	,sm:1.78
step: 417,	,class_loss:0.3931,	,sm:1.67
step: 418,	,class_loss:0.2977,	,sm:1.63
step: 419,	,class_loss:0.3843,	,sm:1.55
step: 420,	,class_loss:0.2572,	,sm:1.69
step: 421,	,class_loss:0.2722,	,sm:1.70
step: 422,	,class_loss:0.4971,	,sm:1.97
step: 423,	,class_loss:0.3471,	,sm:1.79
step: 424,	,class_loss:0.2954,	,sm:1.68
step: 425,	,class_loss:0.3116,	,sm:1.74
step: 426,	,class_loss:0.3256,	,sm:1.71


step: 602,	,class_loss:0.1842,	,sm:1.55
step: 603,	,class_loss:0.1623,	,sm:1.52
step: 604,	,class_loss:0.3118,	,sm:1.51
step: 605,	,class_loss:0.3225,	,sm:1.49
step: 606,	,class_loss:0.1655,	,sm:1.45
step: 607,	,class_loss:0.0914,	,sm:1.34
step: 608,	,class_loss:0.2866,	,sm:1.27
step: 609,	,class_loss:0.2098,	,sm:1.27
step: 610,	,class_loss:0.3468,	,sm:1.28
step: 611,	,class_loss:0.4014,	,sm:1.31
step: 612,	,class_loss:0.2711,	,sm:1.45
step: 613,	,class_loss:0.2027,	,sm:1.33
step: 614,	,class_loss:0.2249,	,sm:1.46
step: 615,	,class_loss:0.2484,	,sm:1.26
step: 616,	,class_loss:0.1270,	,sm:1.27
step: 617,	,class_loss:0.1303,	,sm:1.27
step: 618,	,class_loss:0.1691,	,sm:1.27
step: 619,	,class_loss:0.1632,	,sm:1.41
step: 620,	,class_loss:0.2426,	,sm:1.35
step: 621,	,class_loss:0.1231,	,sm:1.40
step: 622,	,class_loss:0.4098,	,sm:1.62
step: 623,	,class_loss:0.2161,	,sm:1.33
step: 624,	,class_loss:0.3165,	,sm:1.38
step: 625,	,class_loss:0.3000,	,sm:1.40
step: 626,	,class_loss:0.1693,	,sm:1.31


step: 802,	,class_loss:0.1978,	,sm:1.21
step: 803,	,class_loss:0.3068,	,sm:1.32
step: 804,	,class_loss:0.1967,	,sm:1.19
step: 805,	,class_loss:0.0746,	,sm:1.17
step: 806,	,class_loss:0.0689,	,sm:1.19
step: 807,	,class_loss:0.2379,	,sm:1.15
step: 808,	,class_loss:0.1959,	,sm:1.21
step: 809,	,class_loss:0.1841,	,sm:1.25
step: 810,	,class_loss:0.1624,	,sm:1.08
step: 811,	,class_loss:0.2090,	,sm:1.15
step: 812,	,class_loss:0.2796,	,sm:1.22
step: 813,	,class_loss:0.3676,	,sm:1.26
step: 814,	,class_loss:0.2323,	,sm:1.24
step: 815,	,class_loss:0.1964,	,sm:1.09
step: 816,	,class_loss:0.1517,	,sm:1.12
step: 817,	,class_loss:0.1466,	,sm:1.10
step: 818,	,class_loss:0.2412,	,sm:0.98
step: 819,	,class_loss:0.1241,	,sm:1.17
step: 820,	,class_loss:0.2572,	,sm:1.20
step: 821,	,class_loss:0.1218,	,sm:1.11
step: 822,	,class_loss:0.1913,	,sm:1.14
step: 823,	,class_loss:0.2007,	,sm:1.00
step: 824,	,class_loss:0.0995,	,sm:1.06
step: 825,	,class_loss:0.0543,	,sm:1.05
step: 826,	,class_loss:0.1849,	,sm:1.06


...

iter: 01199, 	 precision: 0.8514,	 best_acc:0.8514
step: 1199,	,class_loss:0.1728,	,sm:0.80
step: 1200,	,class_loss:0.1564,	,sm:1.12
step: 1201,	,class_loss:0.0435,	,sm:1.03
step: 1202,	,class_loss:0.1452,	,sm:0.82
step: 1203,	,class_loss:0.0543,	,sm:0.92
step: 1204,	,class_loss:0.0759,	,sm:0.81
step: 1205,	,class_loss:0.0418,	,sm:0.81
step: 1206,	,class_loss:0.0866,	,sm:0.78
step: 1207,	,class_loss:0.0494,	,sm:0.74
step: 1208,	,class_loss:0.1377,	,sm:0.75
step: 1209,	,class_loss:0.0485,	,sm:0.80
step: 1210,	,class_loss:0.0833,	,sm:0.89
step: 1211,	,class_loss:0.0644,	,sm:0.83
step: 1212,	,class_loss:0.0296,	,sm:0.78
step: 1213,	,class_loss:0.0535,	,sm:0.70
step: 1214,	,class_loss:0.0625,	,sm:0.68
step: 1215,	,class_loss:0.1399,	,sm:0.83
step: 1216,	,class_loss:0.0885,	,sm:0.79
step: 1217,	,class_loss:0.0585,	,sm:0.79
step: 1218,	,class_loss:0.0385,	,sm:0.72
step: 1219,	,class_loss:0.1394,	,sm:0.78
step: 1220,	,class_loss:0.0796,	,sm:0.76
step: 1221,	,class_loss:0.0375,	,sm:0.71
step: 

step: 1394,	,class_loss:0.1954,	,sm:0.76
step: 1395,	,class_loss:0.0779,	,sm:0.70
step: 1396,	,class_loss:0.0811,	,sm:0.69
step: 1397,	,class_loss:0.0914,	,sm:0.73
step: 1398,	,class_loss:0.0758,	,sm:0.74
iter: 01399, 	 precision: 0.8594,	 best_acc:0.8594
step: 1399,	,class_loss:0.0775,	,sm:0.75
step: 1400,	,class_loss:0.0415,	,sm:0.71
step: 1401,	,class_loss:0.1364,	,sm:0.66
step: 1402,	,class_loss:0.0643,	,sm:0.72
step: 1403,	,class_loss:0.0380,	,sm:0.79
step: 1404,	,class_loss:0.1220,	,sm:0.75
step: 1405,	,class_loss:0.0262,	,sm:0.78
step: 1406,	,class_loss:0.0515,	,sm:0.69
step: 1407,	,class_loss:0.0765,	,sm:0.62
step: 1408,	,class_loss:0.0579,	,sm:0.67
step: 1409,	,class_loss:0.0746,	,sm:0.68
step: 1410,	,class_loss:0.0773,	,sm:0.70
step: 1411,	,class_loss:0.1240,	,sm:0.73
step: 1412,	,class_loss:0.0751,	,sm:0.83
step: 1413,	,class_loss:0.0541,	,sm:0.73
step: 1414,	,class_loss:0.0902,	,sm:0.68
step: 1415,	,class_loss:0.0222,	,sm:0.70
step: 1416,	,class_loss:0.0452,	,sm:0.67
step: 

step: 1589,	,class_loss:0.0393,	,sm:0.59
step: 1590,	,class_loss:0.1035,	,sm:0.60
step: 1591,	,class_loss:0.0235,	,sm:0.60
step: 1592,	,class_loss:0.0730,	,sm:0.61
step: 1593,	,class_loss:0.0155,	,sm:0.51
step: 1594,	,class_loss:0.0217,	,sm:0.47
step: 1595,	,class_loss:0.0354,	,sm:0.51
step: 1596,	,class_loss:0.0207,	,sm:0.53
step: 1597,	,class_loss:0.0443,	,sm:0.56
step: 1598,	,class_loss:0.0722,	,sm:0.59
iter: 01599, 	 precision: 0.8614,	 best_acc:0.8635
step: 1599,	,class_loss:0.1055,	,sm:0.60
step: 1600,	,class_loss:0.0776,	,sm:0.64
step: 1601,	,class_loss:0.1311,	,sm:0.56
step: 1602,	,class_loss:0.1516,	,sm:0.58
step: 1603,	,class_loss:0.0325,	,sm:0.55
step: 1604,	,class_loss:0.0754,	,sm:0.52
step: 1605,	,class_loss:0.1978,	,sm:0.57
step: 1606,	,class_loss:0.0439,	,sm:0.56
step: 1607,	,class_loss:0.0584,	,sm:0.66
step: 1608,	,class_loss:0.0680,	,sm:0.66
step: 1609,	,class_loss:0.0157,	,sm:0.55
step: 1610,	,class_loss:0.0830,	,sm:0.66
step: 1611,	,class_loss:0.0482,	,sm:0.61
step: 

step: 1784,	,class_loss:0.0150,	,sm:0.57
step: 1785,	,class_loss:0.0452,	,sm:0.56
step: 1786,	,class_loss:0.0263,	,sm:0.62
step: 1787,	,class_loss:0.0624,	,sm:0.61
step: 1788,	,class_loss:0.0118,	,sm:0.69
step: 1789,	,class_loss:0.0666,	,sm:0.57
step: 1790,	,class_loss:0.1742,	,sm:0.57
step: 1791,	,class_loss:0.0730,	,sm:0.51
step: 1792,	,class_loss:0.0267,	,sm:0.51
step: 1793,	,class_loss:0.0490,	,sm:0.50
step: 1794,	,class_loss:0.0353,	,sm:0.64
step: 1795,	,class_loss:0.0347,	,sm:0.59
step: 1796,	,class_loss:0.0382,	,sm:0.58
step: 1797,	,class_loss:0.0412,	,sm:0.59
step: 1798,	,class_loss:0.0525,	,sm:0.56
iter: 01799, 	 precision: 0.8594,	 best_acc:0.8635
step: 1799,	,class_loss:0.0260,	,sm:0.55
step: 1800,	,class_loss:0.0957,	,sm:0.59
step: 1801,	,class_loss:0.0347,	,sm:0.54
step: 1802,	,class_loss:0.0352,	,sm:0.48
step: 1803,	,class_loss:0.0374,	,sm:0.51
step: 1804,	,class_loss:0.0505,	,sm:0.52
step: 1805,	,class_loss:0.0735,	,sm:0.55
step: 1806,	,class_loss:0.0986,	,sm:0.59
step: 

step: 1979,	,class_loss:0.1487,	,sm:0.53
step: 1980,	,class_loss:0.0366,	,sm:0.44
step: 1981,	,class_loss:0.0747,	,sm:0.51
step: 1982,	,class_loss:0.0436,	,sm:0.49
step: 1983,	,class_loss:0.0478,	,sm:0.47
step: 1984,	,class_loss:0.0143,	,sm:0.41
step: 1985,	,class_loss:0.0297,	,sm:0.39
step: 1986,	,class_loss:0.0208,	,sm:0.36
step: 1987,	,class_loss:0.0556,	,sm:0.43
step: 1988,	,class_loss:0.0620,	,sm:0.54
step: 1989,	,class_loss:0.0320,	,sm:0.63
step: 1990,	,class_loss:0.1071,	,sm:0.61
step: 1991,	,class_loss:0.0536,	,sm:0.62
step: 1992,	,class_loss:0.0519,	,sm:0.49
step: 1993,	,class_loss:0.0305,	,sm:0.52
step: 1994,	,class_loss:0.0213,	,sm:0.52
step: 1995,	,class_loss:0.0393,	,sm:0.52
step: 1996,	,class_loss:0.0430,	,sm:0.56
step: 1997,	,class_loss:0.0506,	,sm:0.52
step: 1998,	,class_loss:0.0610,	,sm:0.48
iter: 01999, 	 precision: 0.8614,	 best_acc:0.8635
step: 1999,	,class_loss:0.0365,	,sm:0.43
